### Importe de librerias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
# Carga de datos
df_raw = pd.read_csv('../../data/raw/DATOS_CRUDOS.csv')
df_raw.head()

,ID_ESTACION,FECHA_REPORTE,HORA_REPORTE,TIPO_REPORTE,TEXTO_REPORTE,FECHA_INGRESO,CEDULA,FECHA_HORA_INSERCION,BANDERA
0,392,1900-01-01 23:59:59.000,1900-01-01 23:59:59.000,METAR,SKBQ NIL,2007-05-31 11:39:29.000,NaN,NaN,L
1,392,2006-07-20 00:00:00.000,2006-07-20 00:00:00.000,METAR,SKBQ 200000Z 03009KT 9999 TS FEW015CB SCT080 2...,2006-07-19 19:22:25.000,NaN,NaN,A
2,392,2006-07-20 01:00:00.000,2006-07-20 01:00:00.000,METAR,SKBQ 200100Z 25008KT 9999 TS FEW015CB SCT080 2...,2006-07-19 20:25:49.000,NaN,NaN,A
3,392,2006-07-20 06:00:00.000,2006-07-20 06:00:00.000,METAR,SKBQ 200600Z 29006KT 9999 FEW015 SCT080 26/23 ...,2006-07-20 01:24:29.000,NaN,NaN,A
4,392,2006-07-20 07:00:00.000,2006-07-20 07:00:00.000,METAR,SKBQ 200700Z 32006KT 9999 SCT012 26/24 A2984,2006-07-20 02:26:52.000,NaN,NaN,A


In [10]:
df_raw.shape

(157635, 9)

## Verificar valores nulos en columnas relevantes

In [5]:

df_raw[["FECHA_REPORTE", "HORA_REPORTE", "TEXTO_REPORTE"]].isna().sum()


FECHA_REPORTE    0
HORA_REPORTE     0
TEXTO_REPORTE    0
dtype: int64

### Revisión de registros inválidos

In [14]:
# contar registros inválidos en TEXTO_REPORTE
invalid_count = df_raw["TEXTO_REPORTE"].str.contains("NIL", na=False).sum()
print(f"Número de registros inválidos en TEXTO_REPORTE: {invalid_count}")

Número de registros inválidos en TEXTO_REPORTE: 4791


In [15]:
df_raw = df_raw[~df_raw["TEXTO_REPORTE"].str.contains("NIL", na=False)]
df_raw.shape

(152844, 9)

### Tomar una de las columnas de fecha y hora de reporte, ya que las 2 contienen la misma información}

In [16]:
df_raw["FECHA_HORA_REPORTE"] = pd.to_datetime(df_raw["FECHA_REPORTE"].astype(str))
df_raw.head()

,ID_ESTACION,FECHA_REPORTE,HORA_REPORTE,TIPO_REPORTE,TEXTO_REPORTE,FECHA_INGRESO,CEDULA,FECHA_HORA_INSERCION,BANDERA,FECHA_HORA_REPORTE
1,392,2006-07-20 00:00:00.000,2006-07-20 00:00:00.000,METAR,SKBQ 200000Z 03009KT 9999 TS FEW015CB SCT080 2...,2006-07-19 19:22:25.000,NaN,NaN,A,2006-07-20 00:00:00
2,392,2006-07-20 01:00:00.000,2006-07-20 01:00:00.000,METAR,SKBQ 200100Z 25008KT 9999 TS FEW015CB SCT080 2...,2006-07-19 20:25:49.000,NaN,NaN,A,2006-07-20 01:00:00
3,392,2006-07-20 06:00:00.000,2006-07-20 06:00:00.000,METAR,SKBQ 200600Z 29006KT 9999 FEW015 SCT080 26/23 ...,2006-07-20 01:24:29.000,NaN,NaN,A,2006-07-20 06:00:00
4,392,2006-07-20 07:00:00.000,2006-07-20 07:00:00.000,METAR,SKBQ 200700Z 32006KT 9999 SCT012 26/24 A2984,2006-07-20 02:26:52.000,NaN,NaN,A,2006-07-20 07:00:00
5,392,2006-07-20 08:00:00.000,2006-07-20 08:00:00.000,METAR,SKBQ 200800Z 27004KT 9999 SCT012 25/23 A2984,2006-07-20 03:24:33.000,NaN,NaN,A,2006-07-20 08:00:00


### Conservar sólo columans necesarias

In [20]:
pd.set_option('display.max_colwidth', None)


In [21]:
df_raw = df_raw[["FECHA_HORA_REPORTE", "TEXTO_REPORTE"]]
df_raw.head()

,FECHA_HORA_REPORTE,TEXTO_REPORTE
1,2006-07-20 00:00:00,SKBQ 200000Z 03009KT 9999 TS FEW015CB SCT080 28/24 A2983 CB/TS S
2,2006-07-20 01:00:00,SKBQ 200100Z 25008KT 9999 TS FEW015CB SCT080 27/23 A2985
3,2006-07-20 06:00:00,SKBQ 200600Z 29006KT 9999 FEW015 SCT080 26/23 A2985
4,2006-07-20 07:00:00,SKBQ 200700Z 32006KT 9999 SCT012 26/24 A2984
5,2006-07-20 08:00:00,SKBQ 200800Z 27004KT 9999 SCT012 25/23 A2984


### Separar información de TEXTO_REPORTE en columnas

In [22]:
# Separar tokern iniciales
df_raw["tokens"] = df_raw["TEXTO_REPORTE"].str.split()
df_raw.head()

,FECHA_HORA_REPORTE,TEXTO_REPORTE,tokens
1,2006-07-20 00:00:00,SKBQ 200000Z 03009KT 9999 TS FEW015CB SCT080 28/24 A2983 CB/TS S,"[SKBQ, 200000Z, 03009KT, 9999, TS, FEW015CB, SCT080, 28/24, A2983, CB/TS, S]"
2,2006-07-20 01:00:00,SKBQ 200100Z 25008KT 9999 TS FEW015CB SCT080 27/23 A2985,"[SKBQ, 200100Z, 25008KT, 9999, TS, FEW015CB, SCT080, 27/23, A2985]"
3,2006-07-20 06:00:00,SKBQ 200600Z 29006KT 9999 FEW015 SCT080 26/23 A2985,"[SKBQ, 200600Z, 29006KT, 9999, FEW015, SCT080, 26/23, A2985]"
4,2006-07-20 07:00:00,SKBQ 200700Z 32006KT 9999 SCT012 26/24 A2984,"[SKBQ, 200700Z, 32006KT, 9999, SCT012, 26/24, A2984]"
5,2006-07-20 08:00:00,SKBQ 200800Z 27004KT 9999 SCT012 25/23 A2984,"[SKBQ, 200800Z, 27004KT, 9999, SCT012, 25/23, A2984]"


In [23]:
import re
df_raw["aerodromo"] = df_raw["tokens"].str[0]
df_raw["fecha_zulu"] = df_raw["tokens"].str[1]
df_raw["viento"] = df_raw["tokens"].str[2]
df_raw["visibilidad"] = df_raw["tokens"].str[3]
#extraer temperatura y punto de rocío
df_raw["temperatura/rocio"] = df_raw["TEXTO_REPORTE"].str.extract(r'(\d{2}\/\d{2})')
#presión
df_raw["presion"] = df_raw["TEXTO_REPORTE"].str.extract(r'(A\d{4})')
#nubosidad
df_raw["nubosidad"] = df_raw["TEXTO_REPORTE"].str.findall(r'(FEW\d{3}\w*|SCT\d{3}\w*|BKN\d{3}\w*|OVC\d{3}\w*)').str.join(", ")

#fenómenos meteorológicos
df_raw["fenomenos"] = df_raw["TEXTO_REPORTE"].str.findall(r'\b(TS|RA|SHRA|FG|BR|HZ|CB|VC\w+)\b').str.join(", ")

In [24]:
df_raw[["FECHA_HORA_REPORTE","aerodromo","fecha_zulu","viento","visibilidad","nubosidad","temperatura/rocio","presion","fenomenos"]].head()


,FECHA_HORA_REPORTE,aerodromo,fecha_zulu,viento,visibilidad,nubosidad,temperatura/rocio,presion,fenomenos
1,2006-07-20 00:00:00,SKBQ,200000Z,03009KT,9999,"FEW015CB, SCT080",28/24,A2983,"TS, CB, TS"
2,2006-07-20 01:00:00,SKBQ,200100Z,25008KT,9999,"FEW015CB, SCT080",27/23,A2985,TS
3,2006-07-20 06:00:00,SKBQ,200600Z,29006KT,9999,"FEW015, SCT080",26/23,A2985,
4,2006-07-20 07:00:00,SKBQ,200700Z,32006KT,9999,SCT012,26/24,A2984,
5,2006-07-20 08:00:00,SKBQ,200800Z,27004KT,9999,SCT012,25/23,A2984,


In [25]:
df_procesado = df_raw.drop(columns=["TEXTO_REPORTE", "tokens"])
df_procesado.head()

,FECHA_HORA_REPORTE,aerodromo,fecha_zulu,viento,visibilidad,temperatura/rocio,presion,nubosidad,fenomenos
1,2006-07-20 00:00:00,SKBQ,200000Z,03009KT,9999,28/24,A2983,"FEW015CB, SCT080","TS, CB, TS"
2,2006-07-20 01:00:00,SKBQ,200100Z,25008KT,9999,27/23,A2985,"FEW015CB, SCT080",TS
3,2006-07-20 06:00:00,SKBQ,200600Z,29006KT,9999,26/23,A2985,"FEW015, SCT080",
4,2006-07-20 07:00:00,SKBQ,200700Z,32006KT,9999,26/24,A2984,SCT012,
5,2006-07-20 08:00:00,SKBQ,200800Z,27004KT,9999,25/23,A2984,SCT012,


In [26]:
df_procesado["fenomenos"].value_counts()

fenomenos
                        127314
BR                        5702
VCFG                      4953
CB                        4028
HZ                        3479
                         ...  
VCSH, VCS                    1
RA, VCTS, VCTS               1
RA, CB, VCTS                 1
VCTS, TS                     1
VCTS, VCSH, VCSH, TS         1
Name: count, Length: 132, dtype: int64

In [28]:
#descargar el dataframe procesado
df_procesado.to_csv('../../data/raw/DATOS_PROCESADOS.csv', index=False)